## 1変量データ解析

In [3]:
import numpy as np
import scipy as sp

In [46]:
%precision 3

'%.3f'

In [5]:
fishdata = np.array([2,3,3,4,4,4,4,5,5,6])
fishdata

array([2, 3, 3, 4, 4, 4, 4, 5, 5, 6])

In [6]:
sp.sum(fishdata)

40

### 平均値

In [7]:
N = len(fishdata)
sumvalue=sp.sum(fishdata)
mu = sumvalue / N
mu

4.0

In [8]:
sp.mean(fishdata)

4.0

### 分散

In [9]:
s2 = sp.sum((fishdata - mu) ** 2 ) / N
s2

1.2

In [10]:
sp.var(fishdata,ddof = 0)

1.2

不偏分散の場合はオプションを1

In [11]:
sp.var(fishdata,ddof = 1)

1.3333333333333333

### 標準偏差

In [12]:
sigma = sp.sqrt(sp.var(fishdata,ddof = 1))
sigma

1.1547005383792515

### 標準化

平均を0にする場合、標本を平均値で一律引く

In [13]:
fishdata - mu

array([-2., -1., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  2.])

In [14]:
sp.mean(fishdata-mu)

0.0

標準偏差を1にするには、標本を標準分散で一律割る

In [15]:
fishdata / sigma

array([1.732, 2.598, 2.598, 3.464, 3.464, 3.464, 3.464, 4.33 , 4.33 ,
       5.196])

In [16]:
sp.std(fishdata / sigma,ddof=1)

1.0

両者の特長を持つのが標準化。正規分布に直してるってことでいいのかな
正規分布を用いると外れ値の検出が容易である（後述）

In [17]:
standard = (fishdata - mu) / sigma
standard

array([-1.732, -0.866, -0.866,  0.   ,  0.   ,  0.   ,  0.   ,  0.866,
        0.866,  1.732])

In [18]:
sp.mean(standard)

2.2204460492503132e-17

In [19]:
sp.std(standard, ddof = 1)

1.0

### 最大値・最小値・中央値

In [20]:
sp.amax(fishdata)

6

In [21]:
sp.amin(fishdata)

2

中央値は平均に引っ張られないので外れ値に対して頑強である

In [22]:
sp.median(fishdata)

4.0

### 四分位点

データを昇順に並べ替えたときに下から25%,75%にくる値.正規分布を見る際に重要

In [23]:
from scipy import stats

In [24]:
fishdata_ = np.array([1,2,3,4,5,6,7,8,9])
stats.scoreatpercentile(fishdata_ , 25)

/home/yu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


3.0

In [25]:
stats.scoreatpercentile(fishdata_ , 75)

/home/yu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


7.0

## 多変量データ解析

### グループ別の統計量(groupby関数)

In [26]:
import pandas as pd

In [27]:
fishmulti = pd.DataFrame({
    "species" : np.array(["A","A","A","B","B","B"]),
    "length" : np.array([2,3,4,6,8,10])
})
fishmulti

,species,length
0,A,2
1,A,3
2,A,4
3,B,6
4,B,8
5,B,10


グループごとに統計量を出したいときはgroupby関数を用いる

In [28]:
group = fishmulti.groupby("species")
print(group.mean())

         length
species        
A             3
B             8


In [29]:
print(group.std(ddof=1))

         length
species        
A           1.0
B           2.0


まとめて出力も可能

In [30]:
group.describe()

length                                    
         count mean  std  min  25%  50%  75%   max
species                                           
A          3.0  3.0  1.0  2.0  2.5  3.0  3.5   4.0
B          3.0  8.0  2.0  6.0  7.0  8.0  9.0  10.0

### クロス集計表への変換

In [31]:
shoes = pd.DataFrame({
    "store" : np.array(["Tokyo","Tokyo","Oosaka","Oosaka"]),
    "color" : np.array(["blue","red","blue","red"]),
    "sales" : np.array([10,15,13,9])
})
shoes

,store,color,sales
0,Tokyo,blue,10
1,Tokyo,red,15
2,Oosaka,blue,13
3,Oosaka,red,9


可視化する際はクロス集計表に直したほうが見やすい場合もある
そのとき、pandasのpivot_tale関数を使う

In [32]:
cross = pd.pivot_table(
    data = shoes,
    values = "sales",
    aggfunc = "sum",
    index = "store",
    columns = "color"
)
print(cross)

color   blue  red
store            
Oosaka    13    9
Tokyo     10   15


### 共分散


共分散は2つの連続型変数の関係性を見るときに用いる統計量


分散では変数と平均値の差を2乗（2回かけた）していたが、

変数x,yについてそれぞれの変数の平均値との差をとって掛けた分散の式になる。(x-mu_x)(y-mu_y)

2変数間の共分散が0より大きいとき、変数xが大きな値をとるとき、同様に変数yも大きな値を取る<br>
共分散が0よりも小さいとき、変数xが大きな値を取るとき、変数yは小さな値を取る<br>
共分散が0であれば、両者は独立しており相関性がないといえる

#### 共分散

In [42]:
covdata = pd.DataFrame({
    "x" : sp.array([18.5,18.7,19.1,19.7,21.5,21.7,21.8,22.0,23.4,23.8]),
    "y" : sp.array([34,39,41,38,45,41,52,44,44,49])
})
covdata

,x,y
0,18.5,34
1,18.7,39
2,19.1,41
3,19.7,38
4,21.5,45
5,21.7,41
6,21.8,52
7,22.0,44
8,23.4,44
9,23.8,49


In [34]:
x = covdata["x"]
y = covdata["y"]
N = len(covdata)
mu_x = sp.mean(x)
mu_y = sp.mean(y)

共分散。不偏分散として計算することもある

In [35]:
covsample = sum((x-mu_x)*(y-mu_y)) / N
covsample

6.906

In [44]:
# 不偏共分散
cov = sum((x-mu_x)*(y-mu_y)) / (N-1)
cov

7.673

#### 分散共分散行列(cov関数)

$    \left[ \begin{array}{c} 
\sigma^2_x & Cov(x,y) \\
Cov(x,y) & \sigma^2_y \\ 
\end{array}\right]   $

In [37]:
sp.cov(x,y,ddof=0)

array([[ 3.282,  6.906],
       [ 6.906, 25.21 ]])

In [38]:
sp.cov(x,y,ddof=1)

array([[ 3.646,  7.673],
       [ 7.673, 28.011]])

### ピアソンの積率相関係数 $ \rho_(xy) $

共分散を最大値1,最小値-1で標準化したもの<br>
単に相関係数ともいう<br>
共分散はスケールが統一されていないため、標準化したものをつかうのがふつう<br>
$ \rho_(xy) = Cov(x,y) / \sqrt((\sigma^2_x)(\sigma^2_y))  $

### 相関行列(corrcoef関数)

複数の変数において相関係数の一覧を行列にまとめたもの<br><br>
$    
\left[ \begin{array}{c} 
    1 & \rho_(xy) \\
    \rho_(xy) & 1 \\ 
\end{array}\right]   $

In [49]:
#分散(不偏分散でも結果は変わらない)
sigma_2_x = sp.var(x,ddof=1)
sigma_2_y = sp.var(y,ddof=1)
# 相関係数
rho = cov / sp.sqrt(sigma_2_x * sigma_2_y)
rho

0.7592719041137088

python的にはcorrcoef関数を使う

In [47]:
sp.corrcoef(x,y)

array([[1.   , 0.759],
       [0.759, 1.   ]])

結論として、各々のデータ群にどれほど相関があるのかを示している。<br>
今回の例で言えば、$\boldsymbol{x}と\boldsymbol{y}$の相関度は76%ということ